In [ ]:
import pandas as pd
import argparse
import rdkit

from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
from rdkit import Chem

In [ ]:
rdkit.__version__

'2023.03.3'

In [ ]:
#from molgpt
def check_novelty(gen_smiles, train_smiles): # gen: say 788, train: 120803
    if len(gen_smiles) == 0:
        novel_ratio = 0.
    else:
        duplicates = [1 for mol in gen_smiles if mol in train_smiles]  # [1]*45
        novel = len(gen_smiles) - sum(duplicates)  # 788-45=743
        novel_ratio = novel*100./len(gen_smiles)  # 743*100/788=94.289
    print("novelty: {:.3f}%".format(novel_ratio))
    return novel_ratio

def canonic_smiles(smiles_or_mol):
    mol = get_mol(smiles_or_mol)
    if mol is None:
        return None
    return Chem.MolToSmiles(mol)

    #Experimental Class for Smiles Enumeration, Iterator and SmilesIterator adapted from Keras 1.2.2

In [ ]:
def sca_metrics(data, sca):
    data['mol_scaf'] = data['SMILES'].apply(lambda x: MurckoScaffoldSmiles(x))
    data['fp'] = data['mol_scaf'].apply(lambda x: FingerprintMols.FingerprintMol(Chem.MolFromSmiles(x)))

    max_similarities = []
    for idx, row in data.iterrows():
        max_similarity = 0
        for scaffold in sca:
            cond_fp = FingerprintMols.FingerprintMol(Chem.MolFromSmiles(scaffold))
            similarity = TanimotoSimilarity(row['fp'], cond_fp)
            max_similarity = max(max_similarity, similarity)
        max_similarities.append(max_similarity)

    data['max_similarity'] = max_similarities

    num_rows = data.shape[0]
    count = len(data[data['max_similarity'] == 1])
    print('scafold validity:', count/num_rows*100, '%')

    return data



In [ ]:
#import argparse
import numpy as np
import pandas as pd
import re

#from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torch.cuda.amp import GradScaler

#import rdkit
#import math

import random
#from random import randrange
from random import shuffle

from typing import Optional, List, Tuple

#activation function
from packaging import version
from torch import Tensor

import logging
logger = logging.getLogger(__name__)


# In[2]:


from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import RDLogger

In [ ]:
data = pd.read_csv('/home/10714016/gpt/data_processed.csv', index_col = False)
data = data.dropna(axis=0).reset_index(drop=True)
#data.drop(['SPLIT'], axis=1, inplace=True)

In [ ]:
pattern = "(\[[^\]]+]|<|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
regex = re.compile(pattern)

data['sm_len'] = data['SMILES'].apply(lambda x: len(regex.findall(x.strip())))
data['sca_len'] = data['scaffold_SMILES'].apply(lambda x: len(regex.findall(x.strip())))

max_len = data['sm_len'].max()
sca_max_len = data['sca_len'].max()
print('Max_len:', max_len)
print('Max_len_sca:', sca_max_len)

Max_len: 100
Max_len_sca: 100


In [ ]:
#guacamol
whole_string = ['#', '%10', '%11', '%12', '(', ')', '-', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', '[B-]', '[BH-]', '[BH2-]', '[BH3-]', '[B]', '[C+]', '[C-]', '[CH+]', '[CH-]', '[CH2+]', '[CH2]', '[CH]', '[F+]', '[H]', '[I+]', '[IH2]', '[IH]', '[N+]', '[N-]', '[NH+]', '[NH-]', '[NH2+]', '[NH3+]', '[N]', '[O+]', '[O-]', '[OH+]', '[O]', '[P+]', '[PH+]', '[PH2+]', '[PH]', '[S+]', '[S-]', '[SH+]', '[SH-]', '[SH]', '[Se+]', '[SeH+]', '[SeH]', '[Se]', '[Si-]', '[SiH-]', '[SiH2]', '[SiH]', '[Si]', '[b-]', '[bH-]', '[c+]', '[c-]', '[cH+]', '[cH-]', '[n+]', '[n-]', '[nH+]', '[nH]', '[o+]', '[s+]', '[sH+]', '[se+]', '[se]', 'b', 'c', 'n', 'o', 'p', 's']

char_list = sorted(list(set(whole_string)))

stoi_dict = {'[PAD]': 0, '[BOS]': 1, '[EOS]': 2, '[MASK]': 3}
itos_dict = {}
for i, char in enumerate(char_list):
    stoi_dict[char] = i + 4

itos_dict = {i: char for i, char in enumerate(stoi_dict)}
vocab_size = len(stoi_dict)
vocab_size

98

In [ ]:
class TestDataSet(Dataset):
    #molgpt
    def __init__(self, data, content, prop=None, aug_prob = 0.5):
        chars = sorted(list(set(content)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d smiles, %d unique characters.' % (data_size, vocab_size))

        self.data = data
        self.vocab_size = vocab_size
        self.smiles = data['SMILES']
        self.scaffold = data['scaffold_SMILES']
        #self.prop = prop
        if prop is not None:
            if isinstance(prop, list):
                #a list of properties
                self.prop = {p: data[p] for p in prop}
            else:
                #single property
                self.prop = {prop: data[prop]}
        else:
            self.prop = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        smiles = self.smiles.iloc[idx]  # self.prop.iloc[idx, :].values  --> if multiple properties
        scaffold = self.scaffold.iloc[idx]
        if self.prop is not None:
            prop_values = {key: values.iloc[idx] for key, values in self.prop.items()}
        else:
            prop_values = None

        '''sca = scaffold.strip()
        sca = sca.strip()
        sca = tokenization.tokenize_enc_input(sca)'''
        enc_input = scaffold.strip()
        enc_input = tokenization.tokenize_enc_input(enc_input)
        enc_input_tensor = torch.tensor(enc_input)
        #mask = (torch.tensor(enc_input) == stoi_dict['[MASK]']).float()
        #mask = (enc_input.clone() == stoi_dict['[MASK]']).float()

        #mask = mask.to(torch.float32)
        v_prop_tensor = {key: torch.tensor(value) for key, value in prop_values.items()} if prop_values else None
        #v_prop_tensor = torch.tensor([prop], dtype = torch.float)
        dec_input = smiles.strip()
        dec_input = dec_input.strip()
        dec_input = tokenization.tokenize_dec_input(dec_input)
        dec_input_tensor = torch.tensor(dec_input)

        v_output = smiles.strip()
        v_output = v_output.strip()
        v_output = tokenization.tokenize_v_output(v_output)
        v_output_tensor = torch.tensor(v_output)
        #v_output_tensor =  v_output_tensor.to(torch.float32)
        padding_tensor = (torch.tensor(v_output) != stoi_dict['[PAD]']).float()




        return enc_input_tensor, v_prop_tensor, dec_input_tensor, v_output_tensor, padding_tensor

In [ ]:
class tokenization:
    def tokenize_enc_input(smiles): #for x_tensor & y_tensor
        #batch = []
        smiles_block = regex.findall(smiles)
        tokens = [stoi_dict[s] for s in smiles_block]
        #input_ids = mask_input(tokens)
        input_ids = tokens

        '''if random.random() < 0.5:
            input_ids = mask_input(tokens)
        else:
            #input_ids = generate_inserted_sequence(input_ids, char_list)
            while True:
                input_tokens = tokens.copy()
                input_ids = generate_inserted_sequence(input_tokens, char_list)
                m = detokenize_for_input(input_ids, itos_dict)
                if get_mol(m) is None:
                    break'''
        #Paddings
        n_pad = max_len + 1 - len(input_ids)
        input_ids.extend([0] * n_pad)

        # Zero Padding for masked tokens
        '''if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)'''
            #input_ids_tensor = torch.tensor(dtype=torch.long)
        #batch = [input_ids_tensor, masked_tokens, masked_pos]
        #return batch
        return input_ids
    def tokenize_dec_input(smiles):
        #batch = []
        smiles_block = regex.findall(smiles)
        tokens = [stoi_dict[s] for s in smiles_block]
        input_ids = [stoi_dict['[BOS]']] + tokens


        #Paddings
        n_pad = max_len + 2 - len(input_ids)
        input_ids.extend([0] * n_pad)

            #input_ids_tensor = torch.tensor(dtype=torch.long)
        #batch = [input_ids_tensor, masked_tokens, masked_pos]
        #return batch
        return input_ids
    def tokenize_v_output(smiles):
        #batch = []
        smiles_block = regex.findall(smiles)
        tokens = [stoi_dict[s] for s in smiles_block]
        input_ids = tokens + [stoi_dict['[EOS]']]

        #Paddings
        n_pad = max_len + 2 - len(input_ids)
        input_ids.extend([0] * n_pad)

        return input_ids

In [ ]:
from sklearn.model_selection import train_test_split

training_proportion = 0.7
t_data = data

train_data_o, val_data_o = train_test_split(t_data, test_size=1 - training_proportion, random_state=42)

train_data = TestDataSet(train_data_o, whole_string,
                         prop = ['MW', 'QED', 'SAS', 'TPSA', 'logP'])
train_dataloader = DataLoader(train_data, batch_size=256)

val_data = TestDataSet(val_data_o, whole_string,
                       prop = ['MW', 'QED', 'SAS', 'TPSA', 'logP'])
val_dataloader = DataLoader(val_data, batch_size=256)

data has 2408021 smiles, 94 unique characters.
data has 1032010 smiles, 94 unique characters.


In [ ]:
def get_mol(smiles_or_mol):
    '''
    Loads SMILES/molecule into RDKit's object
    '''
    if isinstance(smiles_or_mol, str):
        if len(smiles_or_mol) == 0:
            return None
        mol = Chem.MolFromSmiles(smiles_or_mol)
        if mol is None:
            return None
        try:
            Chem.SanitizeMol(mol)
        except ValueError:
            return None
        return mol
    return smiles_or_mol

In [ ]:
data = pd.read_csv('/home/10714016/gpt/result/combined_data/df/0/mode.csv')
data

,Unnamed: 0,SMILES,molecule,MW,logP,TPSA,SAS,QED
0,0,N#Cc1ccccc1N1CCN(C(=O)c2cc[nH]c2C#N)CC1,<rdkit.Chem.rdchem.Mol object at 0x7fda588fed50>,305.341,1.72046,86.92,2.573806,0.914878
1,1,N#Cc1ccccc1N1CCN(C(=O)c2cc[nH]c2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7fda588ff370>,280.331,1.84878,63.13,2.277467,0.912784
2,2,Cc1[nH]ccc1C(=O)N1CCN(c2ccc([N+](=O)[O-])cc2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7fda588ff3e0>,314.345,2.19372,82.48,2.288017,0.695213
3,3,N#Cc1ccccc1N1CCN(C(=O)c2cc[nH]c2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7fda588ff450>,280.331,1.84878,63.13,2.277467,0.912784
4,4,Nc1ccc(N)c(N2CCN(C(=O)c3cc[nH]c3)CC2)c1,<rdkit.Chem.rdchem.Mol object at 0x7fda588ff4c0>,285.351,1.14150,91.38,2.415346,0.721601
...,...,...,...,...,...,...,...,...
995,995,N#CC(c1ccccc1)N1CCN(C(=O)c2cc[nH]c2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7fda589ce650>,294.358,2.03738,63.13,2.854336,0.941906
996,996,N#Cc1ccccc1C1CCN(C(=O)c2cc[nH]c2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7fda589ce6c0>,279.343,2.90618,59.89,2.397690,0.918462
997,997,Cc1[nH]ccc1C(=O)N1CCN(c2ccccc2O)CC1,<rdkit.Chem.rdchem.Mol object at 0x7fda589ce730>,285.347,1.99112,59.57,2.281961,0.886881
998,998,N#Cc1ccccc1N1CCN(C(=O)c2cc[nH]c2C#N)CC1,<rdkit.Chem.rdchem.Mol object at 0x7fda589ce7a0>,305.341,1.72046,86.92,2.573806,0.914878


In [ ]:
sca_cond = ['O=C(c1cc[nH]c1)N1CCN(c2ccccc2)CC1']

In [ ]:
data_1 = sca_metrics(data,sca_cond)

scafold validity: 74.1 %


In [ ]:
canon_smiles = [canonic_smiles(s) for s in data['SMILES']]
unique_smiles = list(set(canon_smiles))
novel_ratio = check_novelty(unique_smiles, set(train_data_o['SMILES']))   # replace 'source' with 'split' for moses

#print(f'Condition: {c}')
#print(f'Scaffold: {j}')
#print('Valid ratio: ', np.round(len(data)/(512*gen_iter), 3))
print('Unique ratio: ', np.round(len(unique_smiles)/len(data), 3))
print('Novelty ratio: ', np.round(novel_ratio/100, 3))

novelty: 100.000%
Unique ratio:  0.997
Novelty ratio:  1.0


In [ ]:
generated_smiles = data_1['SMILES'].tolist()
generated_fps = data_1['fp'].tolist()

similarity_matrix = np.zeros((len(generated_smiles), len(generated_smiles)))

for i in range(len(generated_smiles)):
    for j in range(len(generated_smiles)):
        if i != j:
            similarity_matrix[i, j] = AllChem.DataStructs.TanimotoSimilarity(generated_fps[i], generated_fps[j])

int_div_p = np.mean(np.mean(similarity_matrix, axis=1))

print("Internal Diversity (IntDivp):", int_div_p)

Internal Diversity (IntDivp): 0.7383658099140837


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.neighbors import KernelDensity
import numpy as np

reference_smiles = ['CCO', 'CCC', 'CCN']
generated_smiles = ['CCO', 'CCCC', 'CCN']

reference_fps = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 2) for smiles in reference_smiles]
generated_fps = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 2) for smiles in generated_smiles]

reference_array = np.array(reference_fps)
generated_array = np.array(generated_fps)

# Calculate KL Divergence
kl_divergence = np.sum(reference_array * np.log(reference_array / generated_array))

# Calculate Frechet ChemNet Distance
ref_kde = KernelDensity(kernel='gaussian', bandwidth=0.75).fit(reference_array)
gen_kde = KernelDensity(kernel='gaussian', bandwidth=0.75).fit(generated_array)

ref_samples = ref_kde.sample(1000)
gen_samples = gen_kde.sample(1000)

frechet_distance = np.sum((ref_samples - gen_samples)**2)

# Calculate Internal Diversity (IntDivp)
similarity_matrix = np.zeros((len(reference_smiles), len(reference_smiles)))

for i in range(len(reference_smiles)):
    for j in range(len(reference_smiles)):
        similarity_matrix[i, j] = AllChem.DataStructs.TanimotoSimilarity(reference_fps[i], reference_fps[j])

int_div_p = np.mean(np.mean(similarity_matrix, axis=1))

print("KL Divergence:", kl_divergence)
print("Frechet ChemNet Distance:", frechet_distance)
print("Internal Diversity (IntDivp):", int_div_p)
